In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy import sparse
from scipy.sparse import linalg
import bayesian_pdes as bpdes
import scipy
import sympy as sp

In [3]:
l = 1
dt = 0.01
dt_sim = 0.01
omega = 1./15
eta = 0.03
A_0 = 1./30
# static hotspots
theta = 0.56
Gamma = 0.019
# dynamic hotspots
#theta = 5.6
#Gamma = 0.002
z = 2.

D = l**2 / dt
epsilon = theta*dt
gamma = Gamma / l**2
B_bar = epsilon*D*gamma / omega

A_init = A_0 + B_bar
rho_init = B_bar / A_init


In [4]:
x1, x2, y1, y2 = sp.symbols('x_1, x_2, y_1, y_2')
ls = sp.symbols('sigma')
def op_B(k):
    return k - (eta*D/z*(k.diff(x1,x1) + k.diff(x2, x2)) - omega*k)*dt_sim
def op_B_bar(k):
    return k - (eta*D/z*(k.diff(y1,y1) + k.diff(y2, y2)) - omega*k)*dt_sim
def op_B_x1(k):
    return k.diff(x1)
def op_B_x2(k):
    return k.diff(x2)
def op_B_xx(k):
    return k.diff(x1, x1) + k.diff(x2, x2)
def op_B_y1(k):
    return k.diff(y1)
def op_B_y2(k):
    return k.diff(y2)
def op_B_yy(k):
    return k.diff(y1, y1) + k.diff(y2, y2)

def op_rho(k):
    return k - (D*dt_sim/z*(k.diff(x1,x1) + k.diff(x2, x2)))
def op_rho_bar(k):
    return k - (D*dt_sim/z*(k.diff(y1,y1) + k.diff(y2, y2)))
def op_rho_x1(k):
    return k.diff(x1)
def op_rho_x2(k):
    return k.diff(x2)
def op_rho_y1(k):
    return k.diff(y1)
def op_rho_y2(k):
    return k.diff(y2)
#k = 0.001*sp.exp(-(x-y)**2 / (2.*ls**2))
k = 0.001*sp.exp(-2*sp.sin(sp.pi*sp.sqrt((x1-y1)**2)/ 512.)**2 / (2.*ls**2))*sp.exp(-2*sp.sin(sp.pi*sp.sqrt((x2-y2)**2)/ 512.)**2 / (2.*ls**2))
#op_system_B = bpdes.operator_compilation.compile_sympy(
#    [op_B, op_B_x, op_B_xx], 
#    [op_B_bar, op_B_y, op_B_yy], k, [[x], [y], [ls]], 
#    mode='cython'
#)
#op_system_rho = bpdes.operator_compilation.compile_sympy(
#    [op_rho, op_rho_x], 
#    [op_rho_bar, op_rho_y], 
#    k, [[x], [y], [ls]], 
#    mode='cython')

In [7]:
op_system_B = bpdes.operator_compilation.sympy_gram.compile_sympy(
    [op_B, op_B_x1, op_B_x2, op_B_xx], 
    [op_B_bar, op_B_y1, op_B_y2, op_B_yy], 
    k, 
    [[x1, x2], [y1, y2], [ls]], 
    parallel=True,
    limits=[(x1, y1), (x2, y2)]
)

In [9]:
op_system_rho = bpdes.operator_compilation.sympy_gram.compile_sympy(
    [op_rho, op_rho_x1, op_rho_x2], 
    [op_rho_bar, op_rho_y1, op_rho_y2], 
    k, [[x1, x2], [y1, y2], [ls]], 
    parallel=True,
    limits=[(x1, y1), (x2, y2)]
)

In [12]:
N_spatial = 512
step_spatial = 1.
fun_args = np.array([0.01*step_spatial], dtype=np.float) # periodic
#fun_args = np.array([step_spatial], dtype=np.float) # sqexp
x_pts, y_pts = np.mgrid[0:N_spatial:1, 0:N_spatial:1]

In [17]:
design_pts = np.column_stack([x_pts.ravel(), y_pts.ravel()]).astype(np.float)

In [ ]:
posterior_B = bpdes.collocate([op_B], [op_B_bar], [(design_pts, None)], op_system_B, fun_args)
posterior_rho = bpdes.collocate([op_rho], [op_rho_bar], [(design_pts, None)], op_system_rho, fun_args)

interp_posterior_B = bpdes.collocate([()], [()], [(design_pts, None)], op_system_B, fun_args)
interp_posterior_rho = bpdes.collocate([()], [()], [(design_pts, None)], op_system_rho, fun_args)

In [ ]:
time_err_B = bpdes.collocation.compute_operator_matrix([(), op_B_x, op_B_xx], [(), op_B_y, op_B_yy], x_pts, x_pts, op_system_B, fun_args)
time_err_rho = bpdes.collocation.compute_operator_matrix([(), op_rho_x], [(), op_rho_y], x_pts, x_pts, op_system_rho, fun_args)

In [ ]:
posterior_B = posterior_B.apply_operators([(), op_B_x, op_B_xx], [(), op_B_y, op_B_yy])
posterior_rho = posterior_rho.apply_operators([(), op_rho_x], [(), op_rho_y])
interp_posterior_B = interp_posterior_B.apply_operators([(), op_B_x, op_B_xx], [(), op_B_y, op_B_yy])
interp_posterior_rho = interp_posterior_rho.apply_operators([(), op_rho_x], [(), op_rho_y])

In [ ]:
mu_mult_B, cov_B = posterior_B.no_obs_posterior(x_pts)
mu_mult_rho, cov_rho = posterior_rho.no_obs_posterior(x_pts)
interp_mult_B, interp_cov_B = interp_posterior_B.no_obs_posterior(x_pts)
interp_mult_rho, interp_cov_rho = interp_posterior_rho.no_obs_posterior(x_pts)

In [31]:
def safe_sqrt(cov):
    (u, s, v) = np.linalg.svd(cov)
    return np.sqrt(s)[:, None] * v

In [32]:
sqrt_cov_B = safe_sqrt(cov_B)
sqrt_cov_rho = safe_sqrt(cov_rho)
sqrt_interp_cov_B = safe_sqrt(interp_cov_B)
sqrt_interp_cov_rho = safe_sqrt(interp_cov_rho)
sqrt_time_err_B = safe_sqrt(time_err_B)
sqrt_time_err_rho = safe_sqrt(time_err_rho)

In [33]:
def reshape(vec, n_ops):
    ret_tup = ()
    ret_chunks = len(vec) / n_ops
    for i in xrange(n_ops):
        ret_tup += (vec[i*ret_chunks:(i+1)*ret_chunks],)
    return ret_tup

In [34]:
def solve_B(A_n, rho_n, reshape=True):
    rhs_vec = A_n-A_0 + epsilon*D*dt_sim*rho_n*A_n
    ret = np.dot(mu_mult_B, rhs_vec)
    if reshape:
        return reshape(ret, 3)
    return ret
def samp_B(A_n, rho_n, sigma_t):
    mu = solve_B(A_n, rho_n, False)
    ret = mu + np.dot(np.random.normal(size=mu.shape[0]), sqrt_cov_B)
    if sigma_t > 0:
        ret += np.dot(np.random.normal(size=mu.shape[0]), np.sqrt(sigma_t)*sqrt_time_err_B)
    return reshape(ret, 3)

In [35]:
def solve_rho(A_n, rho_n, laplacian_A_n, grad_A_n, grad_rho_n, reshape=True):
    # TODO: need laplacian_A, grad_A, grad_rho
    f = 2*D*dt_sim / z * (laplacian_A_n / A_n - grad_A_n**2 / A_n**2) + A_n*dt_sim
    
    rhs_vec = (1 - f) * rho_n - 2*D*dt_sim/z * grad_rho_n*grad_A_n / A_n + gamma*dt_sim
    
    ret = np.dot(mu_mult_rho, rhs_vec)
    if reshape:
        return reshape(ret, 2)
    return ret
def samp_rho(A_n, rho_n, laplacian_A_n, grad_A_n, grad_rho_n, sigma_t):
    mu = solve_rho(A_n, rho_n, laplacian_A_n, grad_A_n, grad_rho_n, False)
    ret = mu + np.dot(np.random.normal(size=mu.shape[0]), sqrt_cov_rho)
    if sigma_t > 0:
        ret += np.dot(np.random.normal(size=mu.shape[0]), np.sqrt(sigma_t)*sqrt_time_err_rho)
    return reshape(ret, 2)

In [36]:
def solve_forward(A_init, rho_init, times, sigma_t):
    A = []
    rho = []

    A_n, grad_A_n, laplacian_A_n = reshape(np.dot(interp_mult_B, A_init), 3)
    rho_n, grad_rho_n = reshape(np.dot(interp_mult_rho, rho_init), 2)

    A.append(A_n)
    rho.append(rho_n)
    for ix, t in enumerate(times):
        if ix % 1000 == 0: print ix,
        B_next, grad_B_next, laplacian_B_next = samp_B(A_n, rho_n, sigma_t)
        A_next = B_next + A_0

        rho_next, grad_rho_next = samp_rho(A_n, rho_n, laplacian_A_n, grad_A_n, grad_rho_n, sigma_t)

        A.append(A_next)
        rho.append(rho_next)

        A_n = A_next
        grad_A_n = grad_B_next
        laplacian_A_n = laplacian_B_next
        rho_n = rho_next
        grad_rho_n = grad_rho_next
    return A, rho

In [37]:
import pickle

In [38]:
Tend = 200
n_samp = 50
np.random.seed(0)
A_init = A_init*np.ones(len(x_pts)) + np.random.normal(0, 0.01, len(x_pts))
rho_init = rho_init*np.ones(len(x_pts))

skip = int(1/dt_sim)
for i in range(n_samp):
    print i
    A, rho = solve_forward(A_init, rho_init, np.arange(dt_sim, Tend+dt_sim, dt_sim), dt_sim**2)
    with open('crime_results/A_{}.pkl'.format(i), 'wb') as f:
        pickle.dump(A[::skip], f)
    with open('crime_results/rho_{}.pkl'.format(i), 'wb') as f:
        pickle.dump(rho[::skip], f)

0
0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 1
0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 2
0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 3
0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 4
0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 5
0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 6
0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 7
0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 8
0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 9
0 1000 2000 3000 

In [39]:
from tempfile import NamedTemporaryFile

VIDEO_TAG = """<video controls>
 <source src="data:video/x-m4v;base64,{0}" type="video/mp4">
 Your browser does not support the video tag.
</video>"""

def anim_to_html(anim):
    if not hasattr(anim, '_encoded_video'):
        with NamedTemporaryFile(suffix='.mp4') as f:
            anim.save(f.name, fps=20, extra_args=['-vcodec', 'libx264'])
            video = open(f.name, "rb").read()
        anim._encoded_video = video.encode("base64")
    
    return VIDEO_TAG.format(anim._encoded_video)
from matplotlib import animation
animation.Animation._repr_html_ = anim_to_html

In [40]:
# First set up the figure, the axis, and the plot element we want to animate
plt.figure(figsize=(10,10))
ax = plt.gca()
ax.set_xlim(0, N_spatial)
ax.set_ylim(0,1)
fig = plt.gcf()
A_line, = ax.plot([], [], label='A')
rho_line, = ax.plot([], [], label='rho')
skip = int(1/dt_sim)
plt.legend()
# initialization function: plot the background of each frame
def init():
    return []

# animation function.  This is called sequentially
def animate(i):
    A_line.set_data(x_pts, A[skip*i])
    rho_line.set_data(x_pts, rho[skip*i])
    return A_line, rho_line

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=len(A) / skip, interval=20, blit=True)
plt.close()
anim